In [54]:
import librosa
import soundfile
import os
import glob
import pickle
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier #multilayer perceptron-artificial neural networks--several layers of neutrons
from sklearn.metrics import accuracy_score

In [55]:
def extract_feature(file_name, mfcc, chroma, mel):
  with soundfile.SoundFile(file_name) as sound_file:
    X=sound_file.read(dtype="float32")
    sample_rate=sound_file.samplerate
    #If chroma is True, get the Short-Time Fourier Transform of X.
    if chroma:
      stft=np.abs(librosa.stft(X))
    result=np.array([])
    #for each feature of the three, if it exists, make a call to the corresponding function from librosa.feature (eg- librosa.feature.mfcc for mfcc), and get the mean value.
    #hstack() stacks arrays in sequence horizontally
    #mfcc: Mel Frequency Cepstral Coefficient, represents the short-term power spectrum of a sound
    #chroma: Pertains to the 12 different pitch classes
    #mel: Mel Spectrogram Frequency
    if mfcc:
      mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      result=np.hstack((result, mfccs))
    if chroma:
      chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
      result=np.hstack((result, chroma))
    if mel:
      mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
      reslt=np.hstack((result, mel))
  return result

In [56]:
emotions={
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

In [57]:
observed_emotions=['calm','happy','fearful','disgust']

In [58]:
zip_file_path="/content/drive/MyDrive/Classroom/speech-emotion-recognition-ravdess-data.zip"

In [59]:
e_dir="/content/drive/MyDrive/sample_data/emotion_data"

In [60]:
os.makedirs(e_dir,exist_ok=True)

In [61]:
with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
  zip_ref.extractall(e_dir)

In [62]:
ex_files=os.listdir(e_dir)

In [86]:
#glob() function from the glob module to get all the pathnames for the sound files in our dataset

#our function checks whether this emotion is in our list of observed_emotions; if not, it continues to the next file.

def load_data(base_path, test_size=0.2):
  x,y=[],[]
  for file in glob.glob(os.path.join(base_path,'Actor_*','*.wav')):
    file_name=os.path.basename(file)
    emotion=emotions[file_name.split("-")[2]]
    if emotion not in observed_emotions:
      continue
    feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
    x.append(feature)
    y.append(emotion)
  return train_test_split(np.array(x), y, test_size=test_size, random_state=9)


In [87]:
base_path="/content/drive/MyDrive/sample_data/emotion_data"

In [88]:
x_train, x_test, y_train, y_test=load_data(base_path,test_size=0.25)

In [89]:
#Observe the shape of the training and testing datasets:
print((x_train.shape[0],x_test.shape[0]))

(576, 192)


In [90]:
#get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 52


In [91]:
#initialize an MLPClassifier8
model=MLPClassifier(alpha=0.01,batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,),learning_rate='adaptive', max_iter=500)

In [92]:
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [93]:
y_pred=model.predict(x_test)

In [94]:
#call up the accuracy_score() function we imported from sklearn
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

In [95]:
print("Accurcy: {:.2f}%".format(accuracy*100))

Accurcy: 65.10%
